<a href="https://colab.research.google.com/github/mayank-shriv/Alzheimer-s-Disease_detection/blob/main/Alzheimer's_disease.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# ---------------------------------------------
# 📦 STEP 1: IMPORT LIBRARIES
# ---------------------------------------------
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet101, EfficientNetB7, DenseNet201
from keras.applications.resnet50 import preprocess_input as resnet_preprocess
from keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from keras.applications.densenet import preprocess_input as densenet_preprocess
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# ---------------------------------------------
# 📂 STEP 2: EXTRACT DATASETS
# ---------------------------------------------
import zipfile

# Define paths for the zip files
train_zip_path = "/content/train.zip"
test_zip_path = "/content/test.zip"

# Extract the train zip file
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/train")

# Extract the test zip file
with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/test")

# Define paths for the extracted directories
train_path = "/content/train/train"
test_path = "/content/test/test"

# ---------------------------------------------
# ⚙️ STEP 3: IMAGE CONFIG & GENERATORS
# ---------------------------------------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16  # Reduce for memory-intensive models
EPOCHS = 10

def get_data_generators(preprocess_input):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    test_gen = test_datagen.flow_from_directory(
        test_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_gen, test_gen

# ---------------------------------------------
# 🧠 STEP 4: BUILD & TRAIN MODEL (WITH TUNING)
# ---------------------------------------------
def train_with_hyperparams(base_model_class, preprocess_func, dense_units=128, dropout_rate=0.3, learning_rate=1e-4, unfreeze_layers=0):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    if unfreeze_layers > 0:
        base_model.trainable = True
        for layer in base_model.layers[:-unfreeze_layers]:
            layer.trainable = False
    else:
        base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    train_gen, test_gen = get_data_generators(preprocess_func)

    history = model.fit(train_gen, epochs=EPOCHS, validation_data=test_gen, verbose=1)
    return model, history, test_gen

# ---------------------------------------------
# 🔍 STEP 5: EVALUATE AND COMPARE MODELS
# ---------------------------------------------
def evaluate_model(model_name, base_model_class, preprocess_func, configs):
    best_accuracy = 0
    best_model = None
    best_test_gen = None
    best_config = None
    best_preds = None
    best_true_labels = None

    for idx, config in enumerate(configs):
        print(f"\n🔧 Training {model_name} - Config {idx+1}: {config}")
        model, history, test_gen = train_with_hyperparams(
            base_model_class, preprocess_func,
            dense_units=config["dense_units"],
            dropout_rate=config["dropout_rate"],
            learning_rate=config["learning_rate"],
            unfreeze_layers=config["unfreeze_layers"]
        )

        acc = history.history['val_accuracy'][-1]
        print(f"Validation Accuracy: {acc:.4f}")

        if acc > best_accuracy:
            best_accuracy = acc
            best_model = model
            best_test_gen = test_gen
            best_config = config
            preds = best_model.predict(best_test_gen)
            best_preds = np.argmax(preds, axis=1)
            best_true_labels = best_test_gen.classes

    if best_model:
        best_model.save(f"best_model_{model_name.lower()}.keras")
        print(f"\n✅ Best {model_name} model saved with val_accuracy: {best_accuracy:.4f}")
        print(f"Best config: {best_config}")

        print(f"\n📊 {model_name} Classification Report:")
        class_labels = list(best_test_gen.class_indices.keys())
        print(classification_report(best_true_labels, best_preds, target_names=class_labels))

        cm = confusion_matrix(best_true_labels, best_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_labels, yticklabels=class_labels, cmap="Blues")
        plt.title(f"{model_name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.show()

    return model_name, best_accuracy

# ---------------------------------------------
# 🔁 STEP 6: HYPERPARAMETER CONFIGS
# ---------------------------------------------
configs = [
    {
        "dense_units": 2048,         # Maximize representational power
        "dropout_rate": 0.0,         # No regularization — let the model fully fit
        "learning_rate": 1e-3,       # High learning rate for fast adaptation
        "unfreeze_layers": 100       # Fine-tune as much of the base model as possible
    }
]


# ---------------------------------------------
# 🚀 STEP 7: TRAIN & COMPARE ALL MODELS
# ---------------------------------------------
results = []
results.append(evaluate_model("ResNet101", ResNet101, resnet_preprocess, configs))
results.append(evaluate_model("EfficientNetB7", EfficientNetB7, efficientnet_preprocess, configs))
results.append(evaluate_model("DenseNet201", DenseNet201, densenet_preprocess, configs))

# ---------------------------------------------
# 📈 STEP 8: FINAL COMPARISON TABLE
# ---------------------------------------------
print("\n📈 Model Comparison Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")


In [ ]:
# 📈 STEP 8: FINAL COMPARISON TABLE
# ---------------------------------------------
print("\n📈 Model Comparison Results:")
for name, acc in results:
    print(f"{name}: {acc:.4f}")


In [ ]:

# ---------------------------------------------
# 📦 STEP 1: IMPORT LIBRARIES
# ---------------------------------------------
import os
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet101, EfficientNetB7, DenseNet201
from keras.applications.resnet50 import preprocess_input as resnet_preprocess
from keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from keras.applications.densenet import preprocess_input as densenet_preprocess
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# ---------------------------------------------
# 📂 STEP 2: EXTRACT DATASETS
# ---------------------------------------------
train_zip_path = "/content/test.zip"
test_zip_path = "/content/test.zip"

with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/train")

with zipfile.ZipFile(test_zip_path, 'r') as zip_ref:
    zip_ref.extractall("/content/test")

train_path = "/content/train/train"
test_path = "/content/test/test"


In [ ]:
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 16  # Reduce for memory-intensive models
EPOCHS = 5

def get_data_generators(preprocess_input):
    train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        rotation_range=20,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    train_gen = train_datagen.flow_from_directory(
        train_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )

    test_gen = test_datagen.flow_from_directory(
        test_path,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )

    return train_gen, test_gen


In [ ]:
# ---------------------------------------------
# 🧠 STEP 4: BUILD & TRAIN MODEL (WITH TUNING)
# ---------------------------------------------
def train_with_hyperparams(base_model_class, preprocess_func, dense_units=128, dropout_rate=0.3, learning_rate=1e-4, unfreeze_layers=0):
    base_model = base_model_class(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    if unfreeze_layers > 0:
        base_model.trainable = True
        for layer in base_model.layers[:-unfreeze_layers]:
            layer.trainable = False
    else:
        base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(dense_units, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(4, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    train_gen, test_gen = get_data_generators(preprocess_func)

    history = model.fit(train_gen, epochs=EPOCHS, validation_data=test_gen, verbose=1)
    return model, history, test_gen

# ---------------------------------------------

In [ ]:
 #🔍 STEP 5: EVALUATE AND COMPARE MODELS
# ---------------------------------------------
def evaluate_model(model_name, base_model_class, preprocess_func, configs):
    best_accuracy = 0
    best_model = None
    best_test_gen = None
    best_config = None
    best_preds = None
    best_true_labels = None

    for idx, config in enumerate(configs):
        print(f"\n🔧 Training {model_name} - Config {idx+1}: {config}")
        model, history, test_gen = train_with_hyperparams(
            base_model_class, preprocess_func,
            dense_units=config["dense_units"],
            dropout_rate=config["dropout_rate"],
            learning_rate=config["learning_rate"],
            unfreeze_layers=config["unfreeze_layers"]
        )

        acc = history.history['val_accuracy'][-1]
        print(f"Validation Accuracy: {acc:.4f}")

        if acc > best_accuracy:
            best_accuracy = acc
            best_model = model
            best_test_gen = test_gen
            best_config = config
            preds = best_model.predict(best_test_gen)
            best_preds = np.argmax(preds, axis=1)
            best_true_labels = best_test_gen.classes

    if best_model:
        best_model.save(f"best_model_{model_name.lower()}.keras")
        print(f"\n✅ Best {model_name} model saved with val_accuracy: {best_accuracy:.4f}")
        print(f"Best config: {best_config}")

        print(f"\n📊 {model_name} Classification Report:")
        class_labels = list(best_test_gen.class_indices.keys())
        print(classification_report(best_true_labels, best_preds, target_names=class_labels))

        cm = confusion_matrix(best_true_labels, best_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", xticklabels=class_labels, yticklabels=class_labels, cmap="Blues")
        plt.title(f"{model_name} - Confusion Matrix")
        plt.xlabel("Predicted")
        plt.ylabel("True")
        plt.show()

    return model_name, best_accuracy

In [ ]:
# ---------------------------------------------
# 🔁 STEP 6: HYPERPARAMETER CONFIGS
# ---------------------------------------------
configs = [
    {"dense_units": 128, "dropout_rate": 0.3, "learning_rate": 1e-4, "unfreeze_layers": 0},
    {"dense_units": 256, "dropout_rate": 0.5, "learning_rate": 5e-5, "unfreeze_layers": 20},
    {"dense_units": 512, "dropout_rate": 0.4, "learning_rate": 1e-5, "unfreeze_layers": 30},
]


In [ ]:
# ---------------------------------------------
# 🚀 STEP 7: TRAIN & COMPARE ALL MODELS
# ---------------------------------------------
results = []
results.append(evaluate_model("ResNet101", ResNet101, resnet_preprocess, configs))
results.append(evaluate_model("EfficientNetB7", EfficientNetB7, efficientnet_preprocess, configs))
results.append(evaluate_model("DenseNet201", DenseNet201, densenet_preprocess, configs))


in this code we are using hyper parameters tunnung